In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
from scipy.stats import skew,kurtosis

In [2]:
import os

In [5]:
os.chdir('../pickles')

In [6]:
numerical=pickle.load(open('numerical.pickle','rb'))
categorical = pickle.load(open('categorical.pickle','rb'))

In [3]:
os.chdir('../output_data')

In [68]:
df = pd.read_csv('train_v03.csv',header=0)

In [69]:
df.set_index('Id',inplace=True)

In [12]:
def edd(data):
    df_desc = data.describe().transpose()
    df_desc['Var'] = df_desc.index
    df_desc.reset_index(inplace=True)
    df_desc.drop('count',axis=1,inplace=True)
    df_desc['skewness'] = df_desc['Var'].apply(lambda x: skew(np.array(data.loc[data[x].notnull(),x])))
    df_desc['kurtosis'] = df_desc['Var'].apply(lambda x: kurtosis(np.array(data.loc[data[x].notnull(),x]),fisher=False))
    df_desc['99%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.99))
    df_desc['95%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.95))
    df_desc['90%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.90))
    df_desc['10%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.1))
    df_desc['5%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.05))
    df_desc['1%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.01))
    df_desc['mean-3sigma'] = df_desc['mean'] - 3*df_desc['std']
    df_desc['mean+3sigma'] = df_desc['mean'] + 3*df_desc['std']
    df_desc['mean-2sigma'] = df_desc['mean'] - 2*df_desc['std']
    df_desc['mean+2sigma'] = df_desc['mean'] + 2*df_desc['std']
    df_desc['type']='numeric'
    
    def is_category(col):
        return 'float' not in str(data[col].dtype) and 'int' not in str(data[col].dtype)
    
    categorical = [col for col in data.columns if is_category(col)]
    df_categorical = pd.DataFrame()
    df_categorical['Var']=np.array(categorical)
    
    df_categorical['type']='categorical'
    for col in [c for c in df_desc.columns if c not in ['Var','type']]:
        df_categorical[col]=np.nan
    for col in categorical:
        df_var = data[col].value_counts()
        df_cat = pd.DataFrame()
        df_cat['count']=df_var
        df_cat['categories']=df_var.index
        df_cat.reset_index(inplace=True)
        df_cat.sort_values(by='count',ascending=False,inplace=True)
        df_cat.set_index('categories',inplace=True)
        index_list = df_cat.index.tolist()
        for i,c in enumerate(['mean','min','1%','5%','10%','25%']):
            try:
                df_categorical.loc[df_categorical['Var']==col,c] = index_list[i]
            except:
                break
        for i,c in enumerate(['50%','75%','90%','95%','99%','max']):
            try:
                df_categorical.loc[df_categorical['Var']==col,c] = index_list[-(i+1)]
            except:
                break
        del df_var
        del df_cat
        del index_list
    df_categorical = df_categorical[df_desc.columns]
    edd = pd.concat([df_desc,df_categorical])
    del df_desc
    del df_categorical
    edd['count'] = edd['Var'].apply(lambda x: data[data[x].notnull()].shape[0])
    edd['nmiss'] = data.shape[0]-edd['count']
    edd['missing_rate'] = np.array(edd['nmiss']).astype('float')/data.shape[0] * 100
    edd['unique'] = edd['Var'].apply(lambda x: len(data[x].value_counts().index.tolist()))
    orig_cols = ['mean','min','1%','5%','10%','25%','50%','75%','90%','95%','99%','max']
    new_cols = ['mean_or_top1','min_or_top2','p1_or_top3','p5_or_top4','p10_or_top5','p25_or_top6',
                'p50_or_bottom6','p75_or_bottom5','p90_or_bottom4','p95_or_bottom3','p99_or_bottom2','max_or_bottom1']
    
    convert_dict = {}
    for i in range(len(orig_cols)):
        convert_dict[orig_cols[i]]=new_cols[i]
    edd.rename(columns=convert_dict,inplace=True)
    edd = edd[['Var','type','count','nmiss','missing_rate','unique','std','skewness','kurtosis','mean-3sigma',
               'mean-2sigma','mean_or_top1','min_or_top2','p1_or_top3','p5_or_top4','p10_or_top5','p25_or_top6',
               'p50_or_bottom6','p75_or_bottom5','p90_or_bottom4','p95_or_bottom3','p99_or_bottom2','max_or_bottom1'
              ,'mean+2sigma','mean+3sigma']]
    return edd

In [13]:
edd =edd(df)

In [14]:
def correlation(col):
    if col in numerical:
        corr_matrix = df.corr()
        return corr_matrix.loc['SalePrice',col]
    else:
        return np.nan

In [15]:
edd['correlation'] = edd['Var'].apply(lambda x: correlation(x))

In [16]:
os.chdir('../Statistics')

In [17]:
edd.to_csv('edd_after_treatment.csv',index=False)

In [18]:
df.corr()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,MiscFeature_dum_0,MiscFeature_dum_1,MiscFeature_dum_2,SaleType_dum_0,SaleType_dum_1,SaleType_dum_2,SaleType_dum_3,SaleCondition_dum_0,SaleCondition_dum_1,SaleCondition_dum_2
MSSubClass,1.000000,0.012677,-0.412550,-0.074470,-0.043492,-0.035083,-0.007317,0.010651,-0.040947,-0.032647,...,-0.041814,0.055982,-0.008229,0.010836,0.034525,-0.001217,-0.007048,-0.032197,0.009443,-0.002698
LotFrontage,0.012677,1.000000,-0.025938,0.008108,-0.004413,0.012938,0.038055,-0.002107,0.022023,-0.006382,...,0.005731,0.001718,0.000858,0.009452,-0.139880,0.003229,0.001214,0.009920,-0.098367,0.002858
LotArea,-0.412550,-0.025938,1.000000,0.196943,0.012269,0.023636,0.032792,0.015421,0.237117,0.092322,...,0.087006,0.024474,0.034497,0.055612,-0.019806,-0.004959,-0.006005,0.049874,-0.026761,-0.007276
OverallQual,-0.074470,0.008108,0.196943,1.000000,-0.137753,0.527267,0.510709,-0.035088,0.236939,-0.005142,...,-0.086299,-0.044006,0.013127,0.317234,-0.077111,0.019454,-0.047251,0.283107,-0.091951,-0.040703
OverallCond,-0.043492,-0.004413,0.012269,-0.137753,1.000000,-0.429226,0.052435,0.030911,-0.061603,-0.006114,...,0.098007,0.036089,-0.016004,-0.159144,-0.045190,0.011550,-0.038239,-0.166951,-0.050775,-0.032736
YearBuilt,-0.035083,0.012938,0.023636,0.527267,-0.429226,1.000000,0.593707,-0.079572,0.223784,-0.010077,...,-0.065866,-0.033094,0.006346,0.314112,-0.067847,-0.014237,0.002821,0.294097,-0.105170,-0.023661
YearRemodAdd,-0.007317,0.038055,0.032792,0.510709,0.052435,0.593707,1.000000,-0.069469,0.102502,-0.020799,...,-0.054336,-0.045574,-0.010063,0.306454,-0.128146,-0.008134,-0.023183,0.267251,-0.133870,-0.030802
MasVnrArea,0.010651,-0.002107,0.015421,-0.035088,0.030911,-0.079572,-0.069469,1.000000,-0.019033,0.012210,...,0.014049,0.004212,0.002103,-0.058838,0.015037,0.007915,0.002976,-0.054575,0.021381,0.007007
BsmtFinSF1,-0.040947,0.022023,0.237117,0.236939,-0.061603,0.223784,0.102502,-0.019033,1.000000,-0.048146,...,-0.002707,-0.013408,0.022869,0.040494,-0.021679,0.013916,0.025808,0.029781,-0.019156,0.007918
BsmtFinSF2,-0.032647,-0.006382,0.092322,-0.005142,-0.006114,-0.010077,-0.020799,0.012210,-0.048146,1.000000,...,-0.030759,-0.008756,0.021450,-0.045611,0.031554,0.012501,-0.007038,-0.053366,0.051492,-0.016569


In [19]:
def inter_correlation_clusters(data,cutoff=.7):
    correlations=data.corr()
    graph={}
    columns=data.columns
    for i in range(len(columns)):
        graph[i]=[]
        for j in range(len(columns)):
            if i!=j and np.abs(correlations.iloc[i,j])>cutoff:
                graph[i].append(j)
    
    tree_set={}
    component = 0
    visited = [0 for i in range(len(columns))]
    def dfs(i):
        visited[i]=1
        try:
            tree_set[component].append(i)
        except KeyError:
            tree_set[component] = [i]
            
        for j in graph[i]:
            if visited[j]==0:
                dfs(j)
                
    tree_cluster={}
    for key in list(tree_set.keys()):
        tree_cluster[key] = [columns[i] for i in tree_set[key]]
        
    return tree_cluster

In [20]:
def varclus(data):
    columns = []
    correlations=data.corr()
    clusters= inter_correlation_clusters(data)
    print(clusters)
    
    cols = list(data.columns)
    
    def distance(c1,c2):
        return np.max([[np.abs(correlations.loc[i,j]) for i in clusters[c1]] for j in clusters[c2]])
    
    def next_closest(c):
        minima=0
        point=c
        for c1 in [i for i in list(clusters.keys()) if i!=c]:
            dist = distance(c,c1)
            if dist>minima:
                minima=dist
                point=c1
        return point
    
    def get_squared_ratio(col,own_cluster,next_cluster):
        y = np.array(data[col])
        x = np.array(data[own_cluster].drop(col,axis=1))
        model=LinearRegression()
        model.fit(x,y)
        y_pred = list(model.predict(x))
        del x
        del model
        r2_own = r2_score(y,y_pred)
        del y_pred
        x = np.array(data[next_cluster])
        model=LinearRegression()
        model.fit(x,y)
        y_pred = list(model.predict(x))
        del x
        del model
        r2_next = r2_score(y,y_pred)
        del y
        del y_pred
        
        return float(1-r2_own)/(1-r2_next)
    
    for c1 in list(clusters.keys()):
        if len(clusters[c1])>1:
            own_cluster = clusters[c1]
            next_cluster = clusters[next_closest(c1)]
            ration=np.inf
            for col in clusters[c1]:
                col_ratio = get_squared_ratio(col,own_cluster,next_cluster)
                if col_ratio<ratio:
                    ratio=col_ratio
                    clust_col=col
            columns.append(clust_col)
        else:
            columns.append(clusters[c1][0])
            
    return columns

In [21]:
columns = varclus(df.drop(['SalePrice'],axis=1))

{}


In [28]:
for col in df.columns:
    if True in np.isinf(np.array(df[col])):
        print col

# Variance Inflation Factor

In [58]:
df.shape

(1168, 166)

In [27]:
from patsy import dmatrices
import statsmodels
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [70]:
def variance_inflation(data):
    vif = pd.DataFrame()
    vif['features'] = data.columns
    vif['vif factor'] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    vif.sort_values(by='vif factor',ascending=False,inplace=True)
    vif.reset_index(inplace=True)
    vif.drop(['index'],axis=1,inplace=True)
    print(vif)
    return tuple(vif.loc[0,:].values)

In [71]:
vif_drop_cols=[]

In [72]:
def vif_reduction(data,limit=2.5):
    vif = variance_inflation(data)
    if vif[1]<=limit:
        return
    else:
        data.drop(vif[0],axis=1,inplace=True)
        vif_drop_cols.append(vif[0])
        print(vif[0]+' dropped')
        del vif
        vif_reduction(data,limit=limit)

In [73]:
vif_reduction(df.drop(['SalePrice'],axis=1))

                 features    vif factor
0        MasVnrType_dum_2           inf
1             GarageYrBlt  6.868049e+09
2    Utilities_dum_AllPub  2.254263e+02
3             TotalBsmtSF  4.572667e+00
4              GarageCars  4.115945e+00
5              BsmtFinSF1  3.982912e+00
6               BsmtUnfSF  3.855188e+00
7                1stFlrSF  3.393858e+00
8               GrLivArea  3.230451e+00
9               YearBuilt  2.822623e+00
10        ExterCond_dum_2  2.695650e+00
11            OverallQual  2.339259e+00
12         BldgType_dum_2  2.250005e+00
13           YearRemodAdd  2.100039e+00
14               2ndFlrSF  2.011320e+00
15               FullBath  1.978107e+00
16           TotRmsAbvGrd  1.962866e+00
17           BsmtFullBath  1.902620e+00
18         BldgType_dum_0  1.856053e+00
19       HouseStyle_dum_0  1.849900e+00
20             GarageArea  1.743522e+00
21       GarageType_dum_0  1.660863e+00
22        ExterQual_dum_0  1.645408e+00
23         BsmtCond_dum_2  1.598045e+00


                 features  vif factor
0                 MiscVal   19.035208
1               GrLivArea   17.033010
2                PoolArea   16.964434
3       MiscFeature_dum_0   15.867091
4              MSSubClass   15.477710
5              Fireplaces   12.511662
6             TotalBsmtSF   12.320880
7                1stFlrSF   11.161190
8               YearBuilt   10.287827
9          BldgType_dum_2    9.865281
10           PoolQC_dum_0    8.683263
11             BsmtFinSF1    8.343564
12           PoolQC_dum_1    8.300477
13       GarageCond_dum_0    7.846280
14       GarageQual_dum_0    7.801054
15      FireplaceQu_dum_0    7.422368
16         BldgType_dum_0    7.394460
17      FireplaceQu_dum_1    7.365682
18           TotRmsAbvGrd    6.304668
19     BsmtExposure_dum_2    6.120825
20             GarageArea    5.970632
21      Exterior2nd_dum_0    5.752380
22         SaleType_dum_0    5.583653
23      Exterior1st_dum_0    5.445495
24    SaleCondition_dum_0    5.299854
25          

                 features  vif factor
0             TotalBsmtSF   12.154756
1              Fireplaces   12.041255
2               YearBuilt   10.001731
3                1stFlrSF    8.349880
4              BsmtFinSF1    8.228404
5        GarageCond_dum_0    7.812039
6        GarageQual_dum_0    7.747868
7       FireplaceQu_dum_0    7.246087
8       FireplaceQu_dum_1    7.234194
9      BsmtExposure_dum_2    6.085931
10             GarageArea    5.924854
11      Exterior2nd_dum_0    5.736957
12         SaleType_dum_0    5.561556
13           TotRmsAbvGrd    5.448754
14      Exterior1st_dum_0    5.420913
15    SaleCondition_dum_0    5.272280
16             GarageCars    5.213459
17              BsmtUnfSF    5.029900
18            OverallQual    4.782691
19      Exterior1st_dum_2    4.630591
20                LotArea    4.381781
21      Exterior2nd_dum_2    4.263501
22               2ndFlrSF    3.556724
23           YearRemodAdd    3.314944
24               FullBath    3.278735
25       Hou

                 features  vif factor
0                1stFlrSF    6.019050
1              BsmtFinSF1    5.998239
2              GarageArea    5.829322
3       Exterior2nd_dum_0    5.677760
4          SaleType_dum_0    5.512926
5       Exterior1st_dum_0    5.405214
6            TotRmsAbvGrd    5.333153
7     SaleCondition_dum_0    5.260635
8              GarageCars    5.208128
9      BsmtExposure_dum_2    4.894571
10            OverallQual    4.616831
11      Exterior1st_dum_2    4.613926
12                LotArea    4.362370
13      Exterior2nd_dum_2    4.190867
14       GarageQual_dum_0    3.907671
15              BsmtUnfSF    3.620207
16               2ndFlrSF    3.454906
17               FullBath    3.218308
18       Foundation_dum_0    3.155025
19           YearRemodAdd    3.089112
20       HouseStyle_dum_0    3.064078
21         BldgType_dum_0    2.999204
22           BedroomAbvGr    2.949172
23     GarageFinish_dum_1    2.794735
24           KitchenAbvGr    2.689787
25       Gar

                 features  vif factor
0              BsmtFinSF1    4.981235
1      BsmtExposure_dum_2    4.862136
2            TotRmsAbvGrd    4.627944
3             OverallQual    4.542380
4                 LotArea    4.118133
5        GarageQual_dum_0    3.691048
6                2ndFlrSF    3.365613
7                FullBath    3.203965
8        Foundation_dum_0    3.150147
9            YearRemodAdd    3.061798
10      Exterior1st_dum_2    3.047943
11         BldgType_dum_0    2.962215
12           BedroomAbvGr    2.913164
13              BsmtUnfSF    2.799648
14      Exterior2nd_dum_2    2.795924
15     GarageFinish_dum_1    2.769983
16       HouseStyle_dum_0    2.721616
17           KitchenAbvGr    2.683136
18       GarageCond_dum_2    2.618640
19             GarageCars    2.592416
20       Foundation_dum_1    2.571123
21        ExterQual_dum_0    2.567351
22           BsmtFullBath    2.564285
23       GarageType_dum_0    2.533809
24       GarageQual_dum_2    2.504775
25          

                 features  vif factor
0                 LotArea    3.963353
1        GarageQual_dum_0    3.682310
2                2ndFlrSF    3.158701
3        Foundation_dum_0    3.101106
4                FullBath    3.074358
5            YearRemodAdd    3.016710
6       Exterior1st_dum_2    3.001194
7          BldgType_dum_0    2.921097
8       Exterior2nd_dum_2    2.787738
9      GarageFinish_dum_1    2.754867
10       GarageCond_dum_2    2.581607
11       HouseStyle_dum_0    2.542818
12       GarageType_dum_0    2.515687
13       Foundation_dum_1    2.485501
14           KitchenAbvGr    2.483783
15           PoolQC_dum_1    2.483719
16       GarageQual_dum_2    2.481673
17        ExterQual_dum_0    2.469783
18             GarageCars    2.464871
19         BldgType_dum_1    2.435942
20         RoofMatl_dum_0    2.412652
21       GarageType_dum_1    2.410206
22     GarageFinish_dum_0    2.397405
23       CentralAir_dum_Y    2.285366
24        HeatingQC_dum_0    2.285139
25          

                 features  vif factor
0       Exterior1st_dum_2    3.000828
1            YearRemodAdd    2.966231
2       Exterior2nd_dum_2    2.776344
3                FullBath    2.705234
4        GarageCond_dum_2    2.537279
5            KitchenAbvGr    2.477081
6         ExterQual_dum_0    2.435707
7            PoolQC_dum_1    2.405187
8          RoofMatl_dum_0    2.405140
9        GarageQual_dum_2    2.383098
10       HouseStyle_dum_0    2.380077
11         BldgType_dum_1    2.312528
12       GarageType_dum_0    2.291449
13         BldgType_dum_0    2.288517
14             GarageCars    2.288016
15       CentralAir_dum_Y    2.265431
16        HeatingQC_dum_0    2.192244
17         BsmtQual_dum_0    2.117181
18               HalfBath    2.117074
19     GarageFinish_dum_1    2.072972
20      KitchenQual_dum_0    2.004962
21      Exterior1st_dum_3    1.997674
22              BsmtUnfSF    1.995852
23     BsmtFinType1_dum_1    1.990488
24     BsmtFinType1_dum_2    1.986106
25       Fou

                 features  vif factor
0         ExterQual_dum_0    2.421751
1            PoolQC_dum_1    2.381609
2          RoofMatl_dum_0    2.366112
3            KitchenAbvGr    2.313174
4        HouseStyle_dum_0    2.299679
5          BldgType_dum_1    2.269128
6          BldgType_dum_0    2.268961
7        GarageType_dum_0    2.258168
8        CentralAir_dum_Y    2.243530
9              GarageCars    2.210016
10        HeatingQC_dum_0    2.080838
11         BsmtQual_dum_0    2.068352
12     GarageFinish_dum_1    2.042113
13               HalfBath    2.012913
14              BsmtUnfSF    1.992169
15       Foundation_dum_1    1.968693
16     BsmtFinType1_dum_1    1.958272
17      Exterior1st_dum_1    1.925108
18     BsmtFinType1_dum_2    1.924164
19       Condition2_dum_0    1.914783
20      KitchenQual_dum_0    1.909207
21        RoofStyle_dum_1    1.888981
22         BsmtCond_dum_1    1.886429
23        ExterQual_dum_1    1.876740
24         MSZoning_dum_0    1.876438
25       Gar

In [74]:
len(vif_drop_cols)

28

In [76]:
os.chdir('../Statistics')

In [77]:
edd = pd.read_csv('edd_v03.csv',header=0)

In [79]:
edd.loc[edd['Var'].isin(vif_drop_cols),'Status']='drop'

In [80]:
edd['Reason']=''
edd.loc[edd['Var'].isin(vif_drop_cols),'Reason']='VIF'

In [81]:
edd.to_csv('edd_v04.csv',index=False)

In [83]:
df.drop(vif_drop_cols,axis=1,inplace=True)

In [104]:
os.chdir('../output_data')

In [85]:
df.to_csv('train_v04.csv')

# BackWard Selection

In [86]:
import statsmodels.discrete.discrete_model as sm
from sklearn.preprocessing import StandardScaler

In [87]:
import statsmodels.api as sm

In [106]:
flag=0
cols_dropped=['SalePrice']
while flag==0:
    model = sm.OLS(endog=np.array(df['SalePrice']),exog=np.array(df.drop(cols_dropped,axis=1)))
    results = model.fit()
    pvalues=list(results.pvalues)
    drop_index = pvalues.index(max(pvalues))
    col_drop = df.drop(cols_dropped,axis=1).columns[drop_index]
    print(col_drop+'-'+str(pvalues[drop_index]))
    if pvalues[drop_index]>.05:
        cols_dropped.append(col_drop)
    else:
        flag=1

MasVnrArea-0.981068975796
BsmtFinType2_dum_0-0.968705684371
Foundation_dum_2-0.965571710815
HouseStyle_dum_2-0.959763391492
Neighborhood_dum_1-0.934779860187
GarageType_dum_1-0.920425466991
LotConfig_dum_2-0.909446214381
Electrical_dum_2-0.895321807785
RoofStyle_dum_2-0.887558075576
BsmtFinType1_dum_2-0.879087075527
Heating_dum_1-0.810918571501
Condition1_dum_2-0.790047295969
Fence_dum_0-0.781913779847
EnclosedPorch-0.771211271586
Exterior1st_dum_0-0.761559737834
Alley_dum_1-0.752886887353
HeatingQC_dum_2-0.748831470442
BsmtQual_dum_0-0.738645641162
Electrical_dum_0-0.712732749859
YrSold-0.69779802415
KitchenAbvGr-0.64661738871
MoSold-0.620580416397
KitchenQual_dum_2-0.630968947142
ExterQual_dum_2-0.630980884366
GarageFinish_dum_2-0.63218491879
PoolQC_dum_2-0.631164946361
Exterior2nd_dum_4-0.630965618348
HouseStyle_dum_3-0.630965544763
Condition2_dum_3-0.630965523248
LandContour_dum_2-0.630965406476
LotShape_dum_2-0.630965407456
Exterior2nd_dum_2-0.587828810583
BsmtFinType2_dum_1-0.576

In [107]:
cols_dropped.remove('SalePrice')

In [108]:
len(cols_dropped)

76

In [109]:
df.shape

(1168, 138)

In [110]:
df.drop(cols_dropped,axis=1,inplace=True)

In [111]:
df.columns

Index([u'OverallCond', u'BsmtUnfSF', u'BsmtFullBath', u'BsmtHalfBath',
       u'HalfBath', u'BedroomAbvGr', u'GarageCars', u'WoodDeckSF',
       u'OpenPorchSF', u'3SsnPorch', u'ScreenPorch', u'SalePrice',
       u'MSZoning_dum_0', u'MSZoning_dum_1', u'Street_dum_Pave',
       u'LotShape_dum_0', u'LotShape_dum_1', u'LandContour_dum_0',
       u'LandSlope_dum_0', u'LandSlope_dum_1', u'Neighborhood_dum_0',
       u'Neighborhood_dum_2', u'Neighborhood_dum_3', u'Neighborhood_dum_4',
       u'Condition1_dum_0', u'Condition2_dum_0', u'Condition2_dum_2',
       u'BldgType_dum_0', u'BldgType_dum_2', u'HouseStyle_dum_0',
       u'RoofStyle_dum_0', u'RoofMatl_dum_1', u'RoofMatl_dum_2',
       u'Exterior1st_dum_1', u'Exterior1st_dum_3', u'Exterior2nd_dum_3',
       u'MasVnrType_dum_1', u'ExterQual_dum_0', u'ExterQual_dum_1',
       u'ExterCond_dum_2', u'BsmtQual_dum_1', u'BsmtCond_dum_1',
       u'BsmtCond_dum_2', u'BsmtExposure_dum_1', u'BsmtFinType1_dum_0',
       u'BsmtFinType1_dum_1', u'Heatin

In [112]:
os.chdir('../output_data')

In [113]:
df.to_csv('train_v05.csv')

In [114]:
os.chdir('../pickles')

In [115]:
pickle.dump(vif_drop_cols,open('vif_drop_cols.pickle','wb'))

In [116]:
pickle.dump(cols_dropped,open('backward_drop_cols.pickle','wb'))

In [117]:
df.shape

(1168, 62)

In [118]:
df['SalePrice']

Id
39      109000
788     233000
1326     55000
1299    160000
152     372402
78      127000
110     190000
10      118000
997     136500
886     328900
1312    203000
1058    248000
200     274900
293     131000
1267    122000
1366    216000
1405    105000
1101     60000
1406    275000
1246    178000
945     137500
534      39300
804     582933
1223    143000
846     171000
281     228500
833     237000
151     122000
1036     84000
877     132250
         ...  
1420    223000
1240    265900
326      87000
614     147000
139     230000
570     135960
1376    239000
73      185000
602     141000
1331    227000
463      62383
1303    290000
574     170000
415     228000
971     135000
883     178000
378     340000
401     245500
461     263435
5       250000
974     182000
1131    135000
1277    162000
1228    147000
743     179000
253     173000
727     222000
1293    107500
191     315000
92       98600
Name: SalePrice, Length: 1168, dtype: int64

In [119]:
os.chdir('../Statistics')

In [120]:
edd = pd.read_csv('edd_v04.csv',header=0)

In [121]:
edd.loc[edd['Var'].isin(cols_dropped),'Status']='drop'

In [122]:
edd.loc[edd['Var'].isin(cols_dropped),'Reason']='Backward Selection'

In [123]:
edd.to_csv('edd_v05.csv',index=False)

In [124]:
os.chdir('../pickles')

In [126]:
numericals_final = list(set(set(df.columns) & set(numerical)))

In [127]:
len(numericals_final)

12

In [128]:
numericals_final

['3SsnPorch',
 'BsmtUnfSF',
 'ScreenPorch',
 'BsmtFullBath',
 'OpenPorchSF',
 'HalfBath',
 'OverallCond',
 'BsmtHalfBath',
 'SalePrice',
 'GarageCars',
 'BedroomAbvGr',
 'WoodDeckSF']

In [130]:
dummies_final = [col for col in df.columns if 'dum' in col]

In [131]:
len(dummies_final)

50

In [132]:
pickle.dump(numericals_final,open('numerical_final.pickle','wb'))
pickle.dump(dummies_final,open('dummies_final.pickle','wb'))

# PCA

In [14]:
from sklearn.decomposition import PCA

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scale = StandardScaler()

In [17]:
scale.fit(np.array(df.drop(['SalePrice'],axis=1)))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [18]:
pickle.dump(scale,open('scale.pickle','wb'))

In [19]:
x = scale.transform(df.drop(['SalePrice'],axis=1))

In [20]:
df.shape

(1168, 62)

In [21]:
pca = PCA(n_components=61)

In [22]:
pca.fit(x)

PCA(copy=True, iterated_power='auto', n_components=61, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [23]:
variance_cumulative = np.cumsum(pca.explained_variance_ratio_)

In [24]:
for i in range(61):
    if variance_cumulative[i]>=.9:
        n_components=i+1
        break

In [25]:
n_components

47

In [26]:
pca = PCA(n_components=n_components)

In [27]:
pca.fit(x)

PCA(copy=True, iterated_power='auto', n_components=47, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [28]:
pickle.dump(pca,open('pca.pickle','wb'))

In [29]:
df_pca_weights = pd.DataFrame(pca.components_,columns=df.drop(['SalePrice'],axis=1).columns,index=['pc'+str(i)
                                                                                                  for i in range(1,n_components+1)])

In [30]:
df_pca_magnitude_weights = pd.DataFrame(map(np.square,pca.components_),columns=df.drop(['SalePrice'],axis=1).columns,index=['pc'+str(i)
                                                                                                  for i in range(1,n_components+1)])

In [31]:
df_pca_weights

,OverallCond,BsmtUnfSF,BsmtFullBath,BsmtHalfBath,HalfBath,BedroomAbvGr,GarageCars,WoodDeckSF,OpenPorchSF,3SsnPorch,...,FireplaceQu_dum_1,FireplaceQu_dum_2,GarageType_dum_0,GarageType_dum_2,PoolQC_dum_0,PoolQC_dum_1,SaleType_dum_2,SaleCondition_dum_0,SaleCondition_dum_1,SaleCondition_dum_2
pc1,-0.136811,0.158388,0.092586,-0.034619,0.163100,0.009001,0.315230,0.192849,0.251485,0.019387,...,0.072764,0.055031,-0.293186,-0.038062,0.019122,0.030915,-0.013094,0.175042,-0.056249,-0.034261
pc2,-0.042976,0.162880,0.001539,-0.042147,-0.184572,-0.041297,0.096005,-0.035022,-0.027477,0.012225,...,-0.268653,0.080346,0.141975,0.053210,-0.013553,0.028922,-0.012657,0.195071,0.012322,0.034468
pc3,0.099176,-0.228071,0.324749,0.083762,0.015350,0.027541,0.002456,0.147874,-0.033842,0.062782,...,0.168624,0.002742,-0.089042,0.094451,0.108714,0.157181,-0.020603,-0.165620,-0.047008,0.045804
pc4,0.016533,0.020345,-0.226921,0.002691,0.370118,0.401749,0.077898,-0.038270,0.144416,-0.035047,...,0.114675,-0.031352,0.055565,0.054604,0.144380,0.172975,0.008324,0.011093,0.023853,0.043043
pc5,-0.149792,-0.156816,0.083646,-0.034552,0.073553,-0.172319,-0.061120,0.000644,0.020389,-0.028208,...,-0.005273,-0.024320,0.067469,0.055017,-0.055366,-0.040458,0.054184,-0.017898,0.038061,-0.038639
pc6,-0.240307,0.010244,0.070960,0.013231,-0.053531,0.004583,0.036814,-0.004985,-0.065952,-0.018248,...,0.159507,-0.036381,-0.114908,-0.048720,0.254528,0.272294,-0.026982,0.005048,0.041800,0.142092
pc7,0.082581,-0.252553,0.200520,-0.087819,0.101806,-0.056919,-0.093886,0.003896,0.036348,-0.064474,...,0.008040,0.089060,0.138976,-0.109535,0.290155,0.289025,0.011176,-0.051919,-0.005554,0.044771
pc8,0.228294,0.034993,-0.133683,0.174423,0.013160,-0.008510,-0.085349,-0.094045,-0.048315,0.065422,...,-0.186648,0.124936,-0.012277,-0.077810,0.249955,0.298122,0.019038,0.081495,0.062667,-0.049625
pc9,0.115997,0.167345,-0.163036,-0.022315,-0.041443,-0.102844,-0.066846,0.054743,-0.008154,0.081846,...,0.052193,-0.124856,0.033540,-0.228293,-0.138588,-0.009634,-0.049540,-0.023049,-0.074026,-0.073425
pc10,-0.221012,0.095144,-0.001510,-0.104769,-0.203351,0.020923,-0.021081,-0.095345,0.025238,0.044057,...,-0.131211,-0.114277,0.074094,0.120312,0.185946,0.308837,0.035767,0.233410,0.218677,0.256436


In [32]:
os.chdir('../PCA')

In [33]:
df_pca_weights.to_csv('pca_weights.csv')

In [34]:
df_pca_magnitude_weights.to_csv('pca_magnitude_weights.csv')

In [35]:
df_pca = pd.DataFrame(pca.transform(x),columns=['pc'+str(i) for i in range(1,n_components+1)])

In [36]:
df_pca['SalePrice']=df['SalePrice']

In [37]:
os.chdir('../output_data')

In [39]:
df_pca.index = df.index

In [40]:
df_pca

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,...,pc39,pc40,pc41,pc42,pc43,pc44,pc45,pc46,pc47,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
39,-1.885387,-0.468910,-0.022567,-1.215846,-0.045145,-0.066776,0.750570,0.490334,-0.336393,1.306730,...,-0.434359,0.255047,-0.667087,-0.544459,0.343788,1.032445,0.833586,0.134652,-0.829032,NaN
788,1.495878,-1.698501,0.365776,0.634946,-0.423237,0.006943,0.516632,-1.595474,-0.604379,0.280434,...,-0.420270,-1.143630,-0.003843,-0.846904,-0.573658,0.422317,0.191394,0.654612,0.319599,208500.0
1326,-4.758641,2.532141,-2.692268,1.038247,1.688823,4.482673,-1.307318,-2.302366,0.329628,-1.201306,...,-0.974653,0.836818,-1.138235,0.226185,-0.514699,-1.187747,0.905486,0.120118,0.192582,181500.0
1299,5.552815,8.078611,9.406661,6.949376,2.213758,5.791564,5.856534,9.443505,6.025815,8.650497,...,-1.732089,-1.523271,2.615740,2.885541,0.486101,0.397181,1.506087,0.137135,-0.243357,223500.0
152,4.393174,2.832514,-0.039253,-2.243054,-0.109536,0.234127,-0.300298,0.193794,-1.010635,0.981172,...,0.633532,0.377516,-0.861890,-0.161765,-1.862780,0.178446,0.690018,-0.784151,0.353046,140000.0
78,-3.110600,0.074686,0.353283,-2.228528,0.310131,0.036101,1.553493,-0.516714,-0.625673,0.519030,...,0.127545,-0.095973,0.726098,-0.418858,-0.872237,-0.401766,-0.058088,0.405095,0.145410,250000.0
110,1.011247,-0.989898,0.347220,-0.747759,-1.299722,1.158947,-0.187836,-0.742028,0.260022,-0.586533,...,-0.339578,-0.045723,-0.214428,-0.880229,-1.918955,-0.272686,0.325978,-0.941541,-0.812213,143000.0
10,-1.277395,-0.601839,0.960714,-0.281812,0.268612,-0.551595,1.011053,-1.680909,-1.436706,-0.343833,...,2.003481,1.089659,0.962127,0.947397,0.893650,-0.373950,0.835059,-0.608525,0.407737,307000.0
997,-0.923219,-0.200844,0.617834,-0.813206,-1.181669,-0.109981,0.067778,0.367244,-0.030400,0.095178,...,-0.323219,0.986782,-0.994729,-0.520521,-1.177895,0.235785,-0.098344,0.543283,0.072985,200000.0


In [43]:
df_pca['SalePrice']=df['SalePrice']

In [44]:
df_pca.to_csv('train_pca.csv')